<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/Desafio/Desafio_2/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.
Probar términos de interés y explicar
similitudes en el espacio de embeddings.
Intentar plantear y probar tests de
analogías. Graficar los embeddings
resultantes.
Sacar conclusiones.

Algunos de los sitios de textos sugeridos fueron:
- [Project Gutenberg](https://www.gutenberg.org/browse/scores/top)
- [Textos.info](https://www.textos.info/)

Se eligió analizar los textos pertenecientes a [The Project Gutenberg eBook of The Adventures of Sherlock Holmes](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)


In [9]:
import os
filename = 'dataset.txt'
text_path = 'https://raw.githubusercontent.com/jmestanza/natural-language-processing-practice/refs/heads/main/desafios/Desafio_2/the_adventures_of_sherlock_holmes.txt'
curr_dir = os.getcwd()
dataset_path = os.path.join(curr_dir, filename)
if not os.path.exists(dataset_path):
  !wget {text_path} -O {filename}

os.listdir(curr_dir)


['.config', 'dataset.txt', '.ipynb_checkpoints', 'sample_data']

In [15]:
with open(dataset_path, 'r') as f:
  text = f.read()

print(text[:1500])

The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***




The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A C

## Limpieza de dataset
Debemos obtener sólo los textos de adentro de los capítulos y el titulo

In [31]:
import re

# Read the text file
with open(dataset_path, "r", encoding="utf-8") as file:
    text = file.read()

# Remove everything before the actual content
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

# Keep only the content between the start and end markers
text = text.split(start_marker, 1)[-1].split(end_marker, 1)[0]

# Define a pattern to match chapter titles (Roman numerals followed by title)
chapter_pattern = re.compile(r"\n([IVXLCDM]+)\.\s([^\n]+)\n")

# Find all matches (chapter titles)
matches = list(chapter_pattern.finditer(text))

# Split the text into chapters
chapters = []
start = matches[0].end() if matches else 0  # Start at first chapter if found

for i in range(len(matches)):
    end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

    chapter_title = matches[i].group(2)  # Extract chapter title (ignore Roman numeral)
    chapter_content = text[matches[i].end():end].strip()

    # Remove repeated chapter numbers (e.g., "I.") at the start of content
    chapter_content = re.sub(r"^\n?[IVXLCDM]+\.\n+", "", chapter_content).strip()

    # Store as "Chapter Title\nContent"
    chapters.append(f"{chapter_title}\n\n{chapter_content}")

# Print the first 300 characters of each chapter
for i, chapter in enumerate(chapters, 1):
    print(f"Chapter {i}: {chapter[:300]}...\n")


Chapter 1: A SCANDAL IN BOHEMIA

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
wer...

Chapter 2: THE RED-HEADED LEAGUE

I had called upon my friend, Mr. Sherlock Holmes, one day in the
 autumn of last year and found him in deep conversation with a very
 stout, florid-faced, elderly gentleman with fiery red hair. With an
 apology for my intrusion, I was about to withdraw when Holmes pulled
 me a...

Chapter 3: A CASE OF IDENTITY

“My dear fellow,” said Sherlock Holmes as we sat on either side of the
fire in his lodgings at Baker Street, “life is infinitely stranger than
anything which the mind of man could invent. We would not dare to
conceive the things which are really mere commonplaces of existence. If...

Chapter 4: THE BOSCOMBE VALLEY MYSTERY

We were seat

In [37]:
new_text = '\n'.join(chapters)
print(new_text[:1500])

A SCANDAL IN BOHEMIA

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. He
was, I take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in a
false position. He never spoke of the softer passions, save with a gibe
and a sneer. They were admirable things for the observer—excellent for
drawing the veil from men’s motives and actions. But for the trained
reasoner to admit such intrusions into his own delicate and finely
adjusted temperament was to introduce a distracting factor which might
throw a doubt upon all his mental results. Grit in a sensitive
instrument, or a crack in one of his own high-power lenses, would not
be more distu

In [38]:
with open('dataset_clean.txt', 'w') as f:
  f.write(new_text)

Observando el dataset limpio, vemos que
- `\n\n` (consecutive endline)
- `_the_` (emphasis)
- `“Seven!”` (punctuation)
- `“Absolutely?”` (punctuation)
- `John’s` (punctuation)
- `II.` (roman numbers)
- `won’t` (contractions)
- `half-dragged` (hyphened compound-words)
- `“‘The Church of St. Monica,` (abbreviation)
- `\t\t“MY DEAR MR. SHERLOCK HOLMES` (tabs/spaces)
- `_née_` (tilde)
- `“‘Never.’` (quotes)
- `“THE RED-HEADED LEAGUE IS DISSOLVED. October 9, 1890.”` (numbered date)
- `sitting-room`
- `“‘December 22nd. Twenty-four geese at 7_s_. 6_d_.’”`

Deberíamos aplicar un preprocesamiento al texto.

Una práctica recomendada es pasar todo a minúscula

## 1 - Preprocesamiento

Viendo